In [11]:
from jw_package import *

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [12]:
weather = load_weather()

In [13]:
def h_to_m(x):
    length = len(x)
    
    if length==1:
        return np.nan
    
    elif length!=1:
        hour = float(x[:2])
        minute = float(x[2:])
        
        total = 60*hour + minute
        
        return total

In [14]:
weather['sunset'].apply(lambda x: len(x)).unique()

array([1, 4], dtype=int64)

In [15]:
weather['sunrise'].apply(lambda x: len(x)).unique()

array([1, 4], dtype=int64)

In [17]:
weather = load_weather()
weather['sunset'] = weather['sunset'].apply(h_to_m)
weather['sunrise'] = weather['sunrise'].apply(h_to_m)

In [41]:
def missing_to_nan_weather(station_nbr=-1):
    print('missing_to_nan started !')
    
    full_weather = load_weather()
    
    full_weather['sunset'] = full_weather['sunset'].apply(h_to_m)
    full_weather['sunrise'] = full_weather['sunrise'].apply(h_to_m)
    
    if station_nbr != -1:
        full_weather = full_weather[full_weather['station_nbr']==station_nbr]
    
    # sunrise, sunset, category는 연속형변수가 아니므로 일단 뺌
    weathers = set(list(full_weather.columns)[2:20])
    targets = weathers - {'codesum'}
    
    def trim_str(x):
        import re
                            # 실수만 추출
        result = re.findall('(\d+(?:\.\d+)?)',x)
        if result:
            return float(result[0])
        return np.nan
    
    for weather in targets:
        full_weather.loc[:,weather] = full_weather.loc[:,weather].astype(str).apply(trim_str).astype(float)
    
    print('missing_to_nan completed !')
    return full_weather.reset_index(drop=True)

In [42]:
def how_many_missing(nan_weather):
    
    weathers = set(list(nan_weather.columns)[2:20])
    targets = weathers - {'sunset','sunrise','codesum'}
    
    how_many = dict()
    total = len(nan_weather)
    
    for weather in targets:
        not_missing = len(nan_weather[weather].dropna())
    
        msg = 'total = '+ str(total) +' '
        msg += ', missing = '+ str(total - not_missing)
        
        how_many[weather] = msg
    
    return how_many

In [43]:
def all_missing_mean_closest(nan_weather, weather):
    this_station_nbr = nan_weather.loc[0,'station_nbr']
    
    print('\t',weather,' ALL MISSING FEATURE')
    return nan_weather

In [44]:
def replace_by_closest(nan_weather, weather):
    total = len(nan_weather)
    not_missing = len(nan_weather[weather].dropna())
    
    # 1. 아예 없다
    # 다른 station 중 가장 비슷한 feautre를 가진 애
    if not_missing == 0:
        return all_missing_closest(nan_weather, weather)
        
    # 2. 있긴 있다
    # station내에서 가장 비슷한 feature
#     elif 0 < not_missing < total:
#         return partial_missing(nan_weather, weather)
    
    # missing이 없는 경우는 아무것도 안함
    return nan_weather

In [45]:
def drop_all_missing(station_nbr):
    nan_weather = missing_to_nan_weather(station_nbr)
    how_many = how_many_missing(nan_weather)
    
    weathers = set(list(nan_weather.columns)[2:20])
    category = {'codesum'}
    targets = weathers - category
    
    spare_columns = {each : nan_weather[each] for each in list(category)}
    
#     for weather in targets:
#         nan_weather = replace_by_closest(nan_weather,weather)
    
    for each in list(category):
        nan_weather.drop(each, axis=1, inplace=True)
        
    return nan_weather, how_many, spare_columns

In [160]:
def filling_missing_by_closest(station_nbr,precision=-1):
    # 빼기 전 all_missing인 feature,
    # sunset,sunrise,codesum과 같은 비연속 데이터는 제외
    
    weather, how, spare_columns = drop_all_missing(station_nbr)
    full_weather = missing_to_nan_weather()
    
    for each in ['codesum']:
        full_weather.drop(each,axis=1,inplace=True)
    
    # 전체 station에서 intact한 row들
    no_nan = full_weather.dropna(axis=0, how='any').reset_index(drop=True)
    
    if precision!=-1:
        no_nan = no_nan.sample(len(no_nan)//precision).reset_index(drop=True)
    
    # missing이 하나라도 있는 weather의 row들
    weather = weather.reset_index(drop=True)
    nan_index = list(set(weather.index) - set(weather.dropna(axis=0,how='any').index))
    nan = weather.loc[nan_index, :]
    
    print(nan.index)
    
    feature = list(weather.columns)[2:-3]
    table = no_nan.describe()

    print('\t','filling by best_close processing..')

    print(len(no_nan))
    print(len(nan))
    
    tmp=set()
    
    for each in nan.index:
#         if each%100==0:
#             print(each)
        nan_row = nan.loc[each, :]
        nan_z = pd.Series([(nan_row[f] - table[f]['mean']) /
                           table[f]['std'] for f in feature])
        # 얘가 nan이면 그 feature는 missing value임

        distance = dict()

        for idx in no_nan.index:
            intact = no_nan.loc[idx, :]
            intact_z = pd.Series(
                [(intact[f] - table[f]['mean']) / table[f]['std'] for f in feature])

            diff = abs(nan_z - intact_z)
            diff = diff.mean()

            distance[idx] = diff

        no_nan_keys = list(filter(lambda k: not np.isnan(distance[k]), distance))

        # missing인 row가 
        if no_nan_keys:
            best_close_idx = min(distance, key=distance.get)
            missing_feature = nan_row.isnull()
            nan_row[missing_feature] = no_nan.loc[best_close_idx, missing_feature]
            weather.loc[each, :] = nan_row
            tmp.add(each)
        
        else:
            weather.drop(each)
        
    print('\t','filling by best_close finished !')
        
    weather['codesum'] = spare_columns['codesum']
    
    print(set(nan.index) - tmp)
    
    return weather, how

In [161]:
tmp = filling_missing_by_closest(2,100)

missing_to_nan started !
missing_to_nan completed !
missing_to_nan started !
missing_to_nan completed !
Int64Index([   1, 1029,    8,    9,   12,  528,   22,   25,   26,   27,
            ...
             991,  484,  487,  495, 1010,  505, 1018,  507, 1017, 1022],
           dtype='int64', length=217)
	 filling by best_close processing..
76
217


d:\anaconda\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\anaconda\lib\site-packages\pandas\core\generic.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


	 filling by best_close finished !
set()


In [162]:
for station_nbr in range(1,21):
    print('Station number = ',station_nbr)
    
    weather, how = filling_missing_by_closest(station_nbr,100)
    
    file_path = '../data/weather_best_close_final/{:02}.csv'.format(station_nbr)
    
    weather.to_csv(file_path)
    print('\t','write to csv completed !',file_path)
    print('\n')

Station number =  1
missing_to_nan started !
missing_to_nan completed !
missing_to_nan started !
missing_to_nan completed !
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034],
           dtype='int64', length=1035)
	 filling by best_close processing..
76
1035


d:\anaconda\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\anaconda\lib\site-packages\pandas\core\generic.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


	 filling by best_close finished !
{302}
	 write to csv completed ! ../data/weather_best_close_final/01.csv


Station number =  2
missing_to_nan started !
missing_to_nan completed !
missing_to_nan started !
missing_to_nan completed !
Int64Index([   1, 1029,    8,    9,   12,  528,   22,   25,   26,   27,
            ...
             991,  484,  487,  495, 1010,  505, 1018,  507, 1017, 1022],
           dtype='int64', length=217)
	 filling by best_close processing..
76
217
	 filling by best_close finished !
set()
	 write to csv completed ! ../data/weather_best_close_final/02.csv


Station number =  3
missing_to_nan started !
missing_to_nan completed !
missing_to_nan started !
missing_to_nan completed !
Int64Index([ 517,   19,  532,   34,   37,   40,  552,   42,  553,   47,
            ...
             984,  987,  476,  477,  487,  488,  490,  494, 1012,  507],
           dtype='int64', length=156)
	 filling by best_close processing..
76
156
	 filling by best_close finished !
set()
	 wri

d:\anaconda\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars
d:\anaconda\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars


	 filling by best_close finished !
set()
	 write to csv completed ! ../data/weather_best_close_final/07.csv


Station number =  8
missing_to_nan started !
missing_to_nan completed !
missing_to_nan started !
missing_to_nan completed !
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034],
           dtype='int64', length=1035)
	 filling by best_close processing..
76
1035
	 filling by best_close finished !
set()
	 write to csv completed ! ../data/weather_best_close_final/08.csv


Station number =  9
missing_to_nan started !
missing_to_nan completed !
missing_to_nan started !
missing_to_nan completed !
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034],
           dtype='int64', length=1035)
	 filling by best_close processing..
76
1035
	 filling by best_close finished !
{365, 7